In [ ]:
import scipy
import pandas as pd
from pathlib import Path
from utils.classificaton_utils import evaluate_results

# Table and 1.2

In [ ]:
extension = '' # '' or '_lemma'

In [ ]:

results_1850 = evaluate_results(Path(f'results_1850{extension}'))
#results_1900 = evaluate_results(Path(f'results_1900{extension}'))
#results_2000 = evaluate_results(Path('results_2000'))

In [ ]:
df_1850 = pd.DataFrame.from_dict(results_1850, orient='index', columns=['precision','recall','fscore','preds'])
df_1900 = pd.DataFrame.from_dict(results_1900, orient='index', columns=['precision','recall','fscore','preds'])
#df_2000 = pd.DataFrame.from_dict(results_2000, orient='index', columns=['precision','recall','fscore','preds'])


In [ ]:
df_fscores = pd.concat([df_1850['fscore'],df_1900['fscore']],axis=1) # ,df_2000['fscore']
cols_baselines = ["random", "def_tok_overlap_ranking", "sent_embedding", "w2v_lesk_ranking", "svm_wemb_baseline"]
cols_bert = [c for c in df_fscores.index if not 'ts' in c or 'contrast' in c]
df_fscores = df_fscores.loc[cols_baselines + cols_bert]
print(df_fscores.to_latex())

In [ ]:
cols_baselines + cols_bert

In [ ]:
rows_1850 = [r for r in df_fscores.index if '1850' in r]
rows_1900 = [r for r in df_fscores.index if 'blert' in r]
rows_2000 = [r for r in df_fscores.index if 'bert_base' in r]


In [ ]:
diff_1850_2000 = df_1850.loc[df_1850.index.isin(rows_1850)].fscore.values - df_1850.loc[df_1850.index.isin(rows_2000)].fscore.values
diff_1900_2000 = df_1900.loc[df_1900.index.isin(rows_1900)].fscore.values - df_1900.loc[df_1900.index.isin(rows_2000)].fscore.values



# Table 2

In [ ]:
df_diff = pd.DataFrame([diff_1850_2000,diff_1900_2000],columns=rows_2000).T
#df_diff['sum'] = df_diff.sum(axis=1)
print(df_diff.to_latex())

# Table 3

In [ ]:
results_ts = evaluate_results(Path()) # 'results_2000_wo_time_filter'

In [ ]:
df_ts = pd.DataFrame.from_dict(results_ts, orient='index', columns=['precision','recall','fscore'])


In [ ]:
df_ts['fscore']

# Curated examples

In [ ]:
results_curated_seed = evaluate_results(Path(f'results_curated_1920_seed'))
results_curated_synonym = evaluate_results(Path(f'results_curated_1920_syn'))

In [ ]:
df_curated_seed = pd.DataFrame.from_dict(results_curated_seed, orient='index', columns=['precision','recall','fscore'])
df_curated_synonym = pd.DataFrame.from_dict(results_curated_synonym, orient='index', columns=['precision','recall','fscore'])


In [ ]:
df_fscore = pd.concat([df_curated_seed['fscore'],df_curated_synonym['fscore']], axis=1)
df_fscore.columns=['vertical','horizontal']
print(df_fscore.to_latex())

# Assess Statistical Significance

In [ ]:
res = evaluate_results(Path("results_1850/"))

selected = "bert_centroid_sense_vector_blert_base_-1,-2,-3,-4_mean"

selected_pred = res[selected][3][0]
print (selected, res[selected][:3], "\n\nIs the difference significant?\n")

for method,values in res.items():
    if method != selected:
        pred = values[3][0]
        p_value = scipy.stats.ttest_rel(selected_pred,pred)[1]
        if p_value<0.05:
            print (method, values[:3], "YES")
        else:
            print (method, values[:3], "NO p_value:",round(p_value,5))